# Skin lesion Classification and segementation using the pretrianed model RESNET 50 

Importing the library

In [47]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.metrics import confusion_matrix, f1_score



In [52]:
train_image_dir = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\train"
train_metadata_path = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\train_metadata.csv"
val_image_dir = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\val"
val_metadata_path = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\val_metadata.csv"
test_image_dir = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\test"
test_metadata_path = r"C:\Users\Amrit Shah\Desktop\Minor Project\Skin-Lesion-Classification-and-Segmentation\test_metadata.csv"

In [53]:

def load_image_label(image_path, dx):
    try:
        image_data = tf.io.read_file(image_path)
        image_data = tf.image.decode_jpeg(image_data, channels=3)  # Load image
        image_data = tf.image.resize(image_data, [256, 256])       # Ensure it is 256x256
        image_data = tf.image.convert_image_dtype(image_data, tf.float32)  # Normalize to [0, 1]
        dx = tf.cast(dx, tf.int32)
        return image_data, dx
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return tf.zeros([256, 256, 3], dtype=tf.float32), tf.constant(-1, dtype=tf.int32)


In [56]:
def create_dataset(image_dir, metadata_path, batch_size=32, shuffle=True, sample_count=None):
    df = pd.read_csv(metadata_path)

    if sample_count is not None:
        df = df.iloc[:sample_count]

    # Automatically create label mapping
    if df['dx'].dtype == object:
        unique_labels = sorted(df['dx'].unique())
        label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        df['dx'] = df['dx'].map(label_mapping)

    image_paths = df['image_id'].apply(lambda x: os.path.join(image_dir, x + ".jpg")).tolist()
    labels = df['dx'].tolist()
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.map(lambda path, dx: load_image_label(path, dx),
                num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=sample_count or len(df), seed=42)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [57]:

batch_size = 32
train_ds = create_dataset(train_image_dir, train_metadata_path, batch_size=batch_size, shuffle=True, sample_count=12000)
val_ds   = create_dataset(val_image_dir, val_metadata_path, batch_size=batch_size, shuffle=False, sample_count=6000)
test_ds  = create_dataset(test_image_dir, test_metadata_path, batch_size=batch_size, shuffle=False, sample_count=5000)


In [58]:

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # Reduced brightness and contrast augmentation
    image = tf.image.random_brightness(image, max_delta=0.05)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    # Limit rotation to 0° or 90° (instead of 0°, 90°, 180°, 270°)
    k = tf.random.uniform(shape=[], minval=0, maxval=2, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    return image, label

train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)


In [59]:
# Debug: Check a batch's shapes
for img_batch, dx_batch in train_ds.take(1):
    print(f"Image batch shape: {img_batch.shape}")   # Expected: (32, 256, 256, 3)
    print(f"DX batch shape: {dx_batch.shape}")         # Expected: (32,)


Image batch shape: (32, 256, 256, 3)
DX batch shape: (32,)


Load the pretrained model

In [65]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
num_classes=7

# Load pre-trained ResNet50
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(256, 256, 3)
)

# Freeze base layers
base_model.trainable = False

# Add custom head
def resnet(input_shape, num_classes):
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(class_names), activation='softmax')
    ])
    return model

input_shape = (256, 256, 3)
resnet_model = resnet(input_shape, num_classes)
resnet_model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,055 (91.99 MB)

 Trainable params: 526,343 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [66]:

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", SparseCategoricalAccuracy()]
)

In [67]:

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# ---------------------------------------
# Learning Rate Scheduler callback to reduce LR when validation loss plateaus
# ---------------------------------------
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-06)


In [68]:

EPOCHS = 2
history = resnet_model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=EPOCHS, 
    callbacks=[early_stopping, reduce_lr], 
    verbose=1
)

Epoch 1/2
375/375 ━━━━━━━━━━━━━━━━━━━━ 1155s 3s/step - accuracy: 0.4356 - loss: 1.4907 - sparse_categorical_accuracy: 0.4356 - val_accuracy: 0.6360 - val_loss: 0.9705 - val_sparse_categorical_accuracy: 0.6360 - learning_rate: 5.0000e-04
Epoch 2/2
375/375 ━━━━━━━━━━━━━━━━━━━━ 1133s 3s/step - accuracy: 0.6001 - loss: 1.0420 - sparse_categorical_accuracy: 0.6001 - val_accuracy: 0.6607 - val_loss: 0.8892 - val_sparse_categorical_accuracy: 0.6607 - learning_rate: 5.0000e-04
